In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import mysql.connector
from datetime import datetime
from mysql.connector import Error

In [2]:
def conexion_bd():
    
    # Establece la conexión a la base de datos MySQL
    try:
        conexion = mysql.connector.connect(
            host='localhost',
            user='root',
            password='root',
            database='PAD',
            db = "PAD",
            port ='3306'
        )
        if conexion.is_connected():
            cursor = conexion.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print("You're connected to database: ", record)
            return conexion
    except Error as e:
        print("Error while connecting to MySQL", e)

In [3]:
def crear_bd_relacional_mysql():
    # Establece la conexión a la base de datos MySQL
    try:
        conexion = conexion_bd()
        cursor = conexion.cursor()

        # Define la estructura de la tabla Producto
        tabla_producto = """
            CREATE TABLE Producto (
                IDProducto INT AUTO_INCREMENT PRIMARY KEY,
                NombreProducto VARCHAR(255) UNIQUE,
                TipoProducto VARCHAR(255)
        );
        """

        # Define la estructura de la tabla Productos
        tabla_region = """
            CREATE TABLE Region (
                IDRegion INT AUTO_INCREMENT PRIMARY KEY,
                NumeroRegion INT UNIQUE
        );
        """

        tabla_sector = """
            CREATE TABLE Sector (
                IDSector INT AUTO_INCREMENT PRIMARY KEY,
                IDRegion INT,
                NombreSector VARCHAR(255) UNIQUE,
                FOREIGN KEY (IDRegion) REFERENCES Region(IDRegion)
        );
        """

        tabla_monitoreo_precios = """
            CREATE TABLE Monitoreo_precios (
                IDMonitoreo INT AUTO_INCREMENT PRIMARY KEY,
                IDProducto INT,
                IDRegion INT,
                IDSector INT,
                PuntoMonitoreo VARCHAR(255),
                Fecha DATE,
                Semana INT,
                PrecioMinimo DECIMAL(10, 2),
                PrecioMaximo DECIMAL(10, 2),
                PrecioPromedio DECIMAL(10, 2),
                FOREIGN KEY (IDProducto) REFERENCES Producto(IDProducto),
                FOREIGN KEY (IDRegion) REFERENCES Region(IDRegion),
                FOREIGN KEY (IDSector) REFERENCES Sector(IDSector)
        );
        """
        

        # Crea la tabla Monitoreo
        cursor.execute(tabla_producto)
        # Crea la tabla Productos
        cursor.execute(tabla_region)
        # Crea la tabla Sector
        cursor.execute(tabla_sector)
        # Crea la tabla Monitoreo_precios
        cursor.execute(tabla_monitoreo_precios)

        # Confirma los cambios y cierra la conexión
        conexion.commit()
        conexion.close()
        print("Tablas creadas con éxito.")

    except mysql.connector.Error as error:
        print(f"Error: {error}")

In [84]:
def crear_bd_multidimensional_mysql():
    # Establece la conexión a la base de datos MySQL
    try:
        conexion = conexion_bd()
        cursor = conexion.cursor()

        # Define la estructura de la tabla Producto
        tabla_producto = """
            CREATE TABLE Producto (
                IDProducto INT AUTO_INCREMENT PRIMARY KEY,
                NombreProducto VARCHAR(255) UNIQUE,
                TipoProducto VARCHAR(255)
        );
        """

        # Define la estructura de la tabla Productos
        tabla_lugar = """
            CREATE TABLE Lugar (
                IDRegion INT AUTO_INCREMENT PRIMARY KEY,
                PuntoMonitoreo VARCHAR(255) UNIQUE,
                Sector VARCHAR(255) UNIQUE,
                NumeroRegion INT UNIQUE
        );
        """

        tabla_tiempo = """
            CREATE TABLE Tiempo (
                IDTiempo INT AUTO_INCREMENT PRIMARY KEY,
                Fecha DATE,
                Semana INT
        );
        """

        tabla_monitoreo_precios = """
            CREATE TABLE Monitoreo_precios (
                IDMonitoreo INT AUTO_INCREMENT PRIMARY KEY,
                IDProducto INT,
                IDLugar INT,
                PuntoMonitoreo VARCHAR(255),
                PrecioMinimo DECIMAL(10, 2),
                PrecioMaximo DECIMAL(10, 2),
                PrecioPromedio DECIMAL(10, 2),
                FOREIGN KEY (IDProducto) REFERENCES Producto(IDProducto),
                FOREIGN KEY (IDLugar) REFERENCES Region(IDLugar),
                FOREIGN KEY (IDTiempo) REFERENCES Sector(IDTiempo)
        );
        """
        

        # Crea la tabla Monitoreo
        cursor.execute(tabla_producto)
        # Crea la tabla Productos
        cursor.execute(tabla_lugar)
        # Crea la tabla Sector
        cursor.execute(tabla_tiempo)
        # Crea la tabla Monitoreo_precios
        cursor.execute(tabla_monitoreo_precios)

        # Confirma los cambios y cierra la conexión
        conexion.commit()
        conexion.close()
        print("Tablas creadas con éxito.")

    except mysql.connector.Error as error:
        print(f"Error: {error}")

In [ ]:
def cargar_bd_multidimensional_mysql():
    # Establece la conexión a la base de datos MySQL
    try:
        conexion = conexion_bd()
        cursor = conexion.cursor()

        df = pd.read_csv('test.csv', encoding='latin-1')


        for index, row in df.iterrows():
            # Convertir el formato de fecha del CSV ('08/07/2013') a ('YYYY-MM-DD')
            fecha = datetime.strptime(row['Fecha'], '%d/%m/%Y').strftime('%Y-%m-%d')

            # Busca el IDProducto correspondiente en la tabla Producto
            producto_query = "SELECT IDProducto FROM Producto WHERE NombreProducto = %s"
            cursor.execute(producto_query, (row['Producto'],))
            producto_result = cursor.fetchone()
            if producto_result:
                id_producto = producto_result[0]
            else:
                # Si no se encuentra, puedes insertar un nuevo Producto y obtener su ID
                insert_producto_query = "INSERT INTO Producto (NombreProducto, TipoProducto) VALUES (%s, %s)"
                cursor.execute(insert_producto_query, (row['Producto'], row['TipoProducto']))
                id_producto = cursor.lastrowid

            # Busca el IDLugar correspondiente en la tabla Lugar
            lugar_query = "SELECT IDLugar FROM Lugar WHERE PuntoMonitoreo = %s AND Sector = %s AND NumeroRegion = %s"
            cursor.execute(lugar_query, (row['PuntoMonitoreo'], row['Sector'], row['NumeroRegion']))
            lugar_result = cursor.fetchone()
            if lugar_result:
                id_lugar = lugar_result[0]
            else:
                # Si no se encuentra, puedes insertar un nuevo Lugar y obtener su ID
                insert_lugar_query = "INSERT INTO Lugar (PuntoMonitoreo, Sector, NumeroRegion) VALUES (%s, %s, %s)"
                cursor.execute(insert_lugar_query, (row['PuntoMonitoreo'], row['Sector'], row['NumeroRegion']))
                id_lugar = cursor.lastrowid

            # Inserta datos en la tabla Tiempo
            insert_query_tiempo = "INSERT INTO Tiempo (Fecha, Semana) VALUES (%s, %s)"
            data_tiempo = (fecha, row['Semana'])
            cursor.execute(insert_query_tiempo, data_tiempo)

            # Obtiene el IDTiempo insertado
            id_tiempo = cursor.lastrowid

            # Inserta datos en la tabla Monitoreo_precios
            insert_query_monitoreo = """
                INSERT INTO Monitoreo_precios (
                    IDProducto, IDLugar, IDTiempo, PrecioMinimo, PrecioMaximo, PrecioPromedio
                ) VALUES (%s, %s, %s, %s, %s, %s)
            """
            data_monitoreo = (
                id_producto, id_lugar, id_tiempo, row['PrecioMinimo'], row['PrecioMaximo'], row['PrecioPromedio']
            )
            cursor.execute(insert_query_monitoreo, data_monitoreo)

        # Confirma los cambios y cierra la conexión
        conexion.commit()
        conexion.close()
        print("Datos cargados con éxito.")

    except mysql.connector.Error as error:
        print(f"Error: {error}")

In [15]:
def cargar_bd_relacional_mysql():
    try:
        # Establece la conexión a la base de datos MySQL
        conexion = conexion_bd()
        cursor = conexion.cursor()

        # Leer el archivo CSV en un DataFrame de Pandas
        df = pd.read_csv('testPaltas.csv')

        for index, row in df.iterrows():
            # Convertir el formato de fecha del CSV ('08/07/2013') a ('YYYY-MM-DD')
            fecha = datetime.strptime(row['Fecha'], '%d/%m/%Y').strftime('%Y-%m-%d')
            # Busca el IDProducto correspondiente en la tabla Producto
            producto_query = "SELECT IDProducto FROM Producto WHERE NombreProducto = %s"
            cursor.execute(producto_query, (row['Producto'],))
            producto_result = cursor.fetchone()
            if producto_result:
                id_producto = producto_result[0]
            else:
                # Si no se encuentra, puedes insertar un nuevo Producto y obtener su ID
                insert_producto_query = "INSERT INTO Producto (NombreProducto, TipoProducto) VALUES (%s, %s)"
                cursor.execute(insert_producto_query, (row['Producto'], row['TipoProducto']))
                id_producto = cursor.lastrowid

            # Busca el IDRegion correspondiente en la tabla Region
            region_query = "SELECT IDRegion FROM Region WHERE NumeroRegion = %s"
            cursor.execute(region_query, (row['NumeroRegion'],))
            region_result = cursor.fetchone()
            if region_result:
                id_region = region_result[0]
            else:
                # Si no se encuentra, puedes insertar una nueva Región y obtener su ID
                insert_region_query = "INSERT INTO Region (NumeroRegion) VALUES (%s)"
                cursor.execute(insert_region_query, (row['NumeroRegion'],))
                id_region = cursor.lastrowid

            # Busca el IDSector correspondiente en la tabla Sector
            sector_query = "SELECT IDSector FROM Sector WHERE NombreSector = %s AND IDRegion = %s"
            cursor.execute(sector_query, (row['Sector'], id_region))
            sector_result = cursor.fetchone()
            if sector_result:
                id_sector = sector_result[0]
            else:
                # Si no se encuentra, puedes insertar un nuevo Sector y obtener su ID
                insert_sector_query = "INSERT INTO Sector (IDRegion, NombreSector) VALUES (%s, %s)"
                cursor.execute(insert_sector_query, (id_region, row['Sector']))
                id_sector = cursor.lastrowid

            # Inserta datos en la tabla Monitoreo_precios
            insert_query_monitoreo = """
                INSERT INTO Monitoreo_precios (
                    IDProducto, IDRegion, IDSector, PuntoMonitoreo, Fecha, Semana,
                    PrecioMinimo, PrecioMaximo, PrecioPromedio
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            data_monitoreo = (
                id_producto, id_region, id_sector, row['PuntoMonitoreo'], fecha,
                row['Semana'], row['PrecioMinimo'], row['PrecioMaximo'], row['PrecioPromedio']
            )
            cursor.execute(insert_query_monitoreo, data_monitoreo)



        # Confirma los cambios y cierra la conexión
        conexion.commit()
        conexion.close()
        print("Datos cargados con éxito.")

    except mysql.connector.Error as error:
        print(f"Error: {error}")

# Ejecucion de funciones